A notebook to process results of the compression task on CelebA where the input set contains three images, weighted in various ways.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

#%config InlineBackend.figure_format = 'svg'
#%config InlineBackend.figure_format = 'pdf'

In [ ]:
import kbrgan
import kbrgan.kernel as kernel
import kbrgan.glo as glo
import kbrgan.main as main
import kbrgan.net.net as net
import kbrgan.gen as gen
import kbrgan.plot as plot
import kbrgan.embed as embed
import kbrgan.util as util

import matplotlib
import matplotlib.pyplot as plt
import os
import numpy as np
import scipy.stats as stats
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:
# font options
font = {
    #'family' : 'normal',
    #'weight' : 'bold',
    'size'   : 18
}

plt.rc('font', **font)
plt.rc('lines', linewidth=2)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
use_cuda = True and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
tensor_type = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
torch.set_default_tensor_type(tensor_type)

## Plot results

In [ ]:
case_name = 'interpolation_v1'
root_results_folder = glo.result_folder('3imgs_compression', case_name)

Get the input images

In [ ]:
os.path.isdir('/is/ei/wittawat/results/kbrgan/3imgs_compression/interpolation_v1/face_interpolation_test_0_0_1/')

In [ ]:
import glob
import skimage
    
def get_np_input_images(root_results_folder):
    """
    Return a numpy stack of input images. 
    """
    subfolders = glob.glob(os.path.join(root_results_folder, 'face_interpolation_test_*'))
    subfolder2 = glob.glob(subfolders[0] + '/*')
    input_folder = os.path.join(subfolder2[0], 'input_images')
    input_fpaths = glob.glob(input_folder + '/*')
    arg = np.argsort([os.path.basename(fp) for fp in input_fpaths])
    input_fpaths = [input_fpaths[i] for i in arg]
    list_imgs = []
    for img_fpath in input_fpaths:
        img = skimage.io.imread(img_fpath)
#         print(img.shape)
        list_imgs.append(img)
    return np.stack(list_imgs, axis=0)

In [ ]:
import skimage
in_imgs = get_np_input_images(root_results_folder)
for i in range(in_imgs.shape[0]):
    plt.figure()
    plt.imshow(in_imgs[i])

In [ ]:
import re
def get_candidate_weights(root_results_folder):
    """
    Return a numpy array of size n x 3 (of type string) where 
    n is the number of candidate input weight vectors. 
    """
    case_folders = glob.glob(os.path.join(root_results_folder, 'face_interpolation_test*'))
    list_weights = []
#     pat = r'(\d+(\.\d+)?)_(\d+(\.\d+)?)_(\d(\.\d+)?)\.sh'
    pat = r'face_interpolation_test_(\d+(\.\d+)?)_(\d+(\.\d+)?)_(\d(\.\d+)?)'
    for shf in case_folders:
        sh_fname = os.path.basename(shf)
#         print(sh_fname)
        m = re.match(pat, sh_fname)
#         print(m.group(3))
        w = np.array([m.group(i) for i in [1,3,5]])
        list_weights.append(w)
    return np.stack(list_weights, axis=0)

In [ ]:
np_str_weights = get_candidate_weights(root_results_folder)
np_str_weights

In [ ]:
def get_output_img(root_results_folder, weights, verbose=False):
    """
    Return the output image (assume there is only one) given 
    one vector of input weights.
    
    * weights: a numpy array with one dimension. Each value is a string.
    """
    subfolder = 'face_interpolation_test_{}_{}_{}'.format(*weights)
    case_path = os.path.join(root_results_folder, subfolder )
    subfolders = glob.glob(case_path)
    subfolder2 = glob.glob(subfolders[0] + '/*')
    
    output_folder = os.path.join(subfolder2[0], 'output_images')
    iter_fpaths = glob.glob(output_folder + '/*')
#     print([os.path.basename(fname) for fname in iter_fpaths])
    arg = np.argsort([int(os.path.basename(fname)) for fname in iter_fpaths])
    # highest iteration
    last_iter_folder = iter_fpaths[arg[-1]]
    out_fpaths = glob.glob(last_iter_folder + '/*')
    # expect only one output image
    assert len(out_fpaths) == 1
    
    # load the image
    if verbose:
        print('Loading output image: {}'.format(out_fpaths[0]))
    return skimage.io.imread(out_fpaths[0])
    

In [ ]:
# Test get_output_img()
img = get_output_img(root_results_folder, np_str_weights[0])
plt.imshow(img)

In [ ]:
# load all output images. Each output image corresponds to one input weight vector
list_out_imgs = []
for i, weights in enumerate(np_str_weights):
    loaded = get_output_img(root_results_folder, weights)
    list_out_imgs.append(loaded)
    
# stack
out_imgs = np.stack(list_out_imgs, axis=0)

In [ ]:
# show input images
print('Input images')
for i in range(in_imgs.shape[0]):
    plt.figure()
    plt.imshow(in_imgs[i])

In [ ]:
for i in range(out_imgs.shape[0]):
    oimg = out_imgs[i]
    plt.figure()
    plt.imshow(oimg)
    Wi = np_str_weights[i]
    plt.title('in weights: {}, {}, {}'.format(Wi[0], Wi[1], Wi[2]))